In [ ]:
# Librerias.

import os
import pandas as pd
from netCDF4 import Dataset
import numpy as np

En orden de facilitar el cálculo de la clorofila, automatizaremos el proceso para actuar directamente sobre los archivos en la carpeta destino.

In [1]:
# Definir la función.
def recortar_datos_y_guardar_csv(archivo_nc, carpeta_salida):
    # Abrir el archivo.
    archivo = Dataset(archivo_nc, 'r')

    # Obtener variables de latitud, longitud y datos.
    latitudes = archivo.variables['lat'][:]
    longitudes = archivo.variables['lon'][:]
    datos = archivo.variables['Chl_a'][:]

    # Definir las coordenadas del Mar de Alborán.
    lat_min, lat_max = 35, 37
    lon_min, lon_max = -6, -1

    # Encontrar índices de las coordenadas del Mar de Alborán.
    lat_inicio = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0][0]
    lat_fin = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[-1][-1]
    lon_inicio = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0][0]
    lon_fin = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[-1][-1]

    # Asegurarse de que los índices sean válidos.
    lat_inicio = max(lat_inicio, 0)
    lat_fin = min(lat_fin, len(latitudes) - 1)
    lon_inicio = max(lon_inicio, 0)
    lon_fin = min(lon_fin, len(longitudes) - 1)

    # Crear una lista de tuplas de longitud, latitud y dato correspondiente.
    datos_alboran = []
    for i in range(lat_inicio, lat_fin + 1):
        for j in range(lon_inicio, lon_fin + 1):
            datos_alboran.append((longitudes[j], latitudes[i], datos[i, j]))

    # Crear DataFrame a partir de la lista de tuplas.
    df = pd.DataFrame(datos_alboran, columns=['Longitud', 'Latitud', 'Chl_a'])

    # Obtener el nombre del archivo sin la extensión y extraer el mes y el año.
    nombre_archivo = os.path.splitext(os.path.basename(archivo_nc))[0]
    mes = nombre_archivo.split('_')[-2]
    ano = nombre_archivo.split('_')[-1]

    # Guardar DataFrame como archivo CSV.
    nombre_csv = f"Clorofila_Alboran_{ano}_{mes}.csv"
    ruta_salida = os.path.join(carpeta_salida, nombre_csv)
    df.to_csv(ruta_salida, index=False)

    # Cerrar el archivo.
    archivo.close()

    print(f"Archivo '{nombre_archivo}' procesado y guardado como '{nombre_csv}' en '{carpeta_salida}'.")


# Ruta de la carpeta de origen de los datos.
carpeta_origen = r'ruta a carpeta clorofila_satelital_alboran'

# Ruta de la carpeta de salida
carpeta_salida = r'ruta a la carpeta de salida (Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV)'

# Obtener la lista de archivos .nc en la carpeta de origen.
archivos_nc = [os.path.join(carpeta_origen, archivo) for archivo in os.listdir(carpeta_origen) if archivo.endswith('.nc')]

# Recorrer cada archivo .nc y aplicar la función.
for archivo_nc in archivos_nc:
    recortar_datos_y_guardar_csv(archivo_nc, carpeta_salida)


Archivo 'GMIS_A_CHLA_01_2003' procesado y guardado como 'Clorofila_Alboran_2003_01.csv' en 'C:\Users\pejer\Desktop\BOOTCAMP\Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV'.
Archivo 'GMIS_A_CHLA_01_2004' procesado y guardado como 'Clorofila_Alboran_2004_01.csv' en 'C:\Users\pejer\Desktop\BOOTCAMP\Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV'.
Archivo 'GMIS_A_CHLA_01_2005' procesado y guardado como 'Clorofila_Alboran_2005_01.csv' en 'C:\Users\pejer\Desktop\BOOTCAMP\Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV'.
Archivo 'GMIS_A_CHLA_01_2006' procesado y guardado como 'Clorofila_Alboran_2006_01.csv' en 'C:\Users\pejer\Desktop\BOOTCAMP\Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV'.
Archivo 'GMIS_A_CHLA_01_2007' procesado y guardado como 'Clorofila_Alboran_2007_01.csv' en 'C:\Users\pejer\Desktop\BOOTCAMP\Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV'.
Archivo 'GMIS_A_CHLA_01_2008' procesado y guardado como 'Clorofila_Alboran_2008_01.cs

A continuación, combinamos los csv de salida.

In [ ]:
# Lista para almacenar los DataFrames de cada archivo CSV.
dfs = []

# Ruta de la carpeta que contiene los archivos CSV.
carpeta_datos = r'ruta a la carpeta (Analisis_Mediterraneo\Datos\Oceanograficos\Clorofila\CSV)'

# Leer todos los archivos CSV en la carpeta y combinarlos en un solo DataFrame.
for archivo in os.listdir(carpeta_datos):
    if archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta_datos, archivo)
        df = pd.read_csv(ruta_archivo)
        # Agregar el nombre de archivo como una columna para extraer año y mes más tarde
        df['Nombre de archivo'] = archivo
        dfs.append(df)

# Combinar todos los DataFrames en uno solo.
df_completo = pd.concat(dfs, ignore_index=True)

# Extraer la información de año y mes del nombre de cada archivo CSV.
df_completo['Año'] = df_completo['Nombre de archivo'].str.split('_').str[-2]
df_completo['MesID'] = df_completo['Nombre de archivo'].str.split('_').str[-1].str.split('.').str[0].astype(int)

# Definir un diccionario para mapear el número del mes al nombre del mes.
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
         7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}

# Mapear el número del mes al nombre del mes.
df_completo['Mes'] = df_completo['MesID'].map(meses)

# Reordenar las columnas.
df_final = df_completo[['Año', 'Mes', 'MesID', 'Chl_a', 'Latitud', 'Longitud']]

# Restringir la longitud y latitud a 4 dígitos máximos.
df_final['Latitud'] = df_final['Latitud'].apply(lambda x: "{:.4f}".format(round(x, 4)))
df_final['Longitud'] = df_final['Longitud'].apply(lambda x: "{:.4f}".format(round(x, 4)))

# Guardar el resultado como csv.
df_final.to_csv('clorofila.csv')